In [ ]:
import os
import pathlib as pl
import platform
import shutil

import flopy
import hvplot.xarray  # noqa
import jupyter_black
import numpy as np
import pywatershed as pws
import xarray as xr
from modflowapi import ModflowApi

from helpers import get_mf6_nightly_build

jupyter_black.load()
pws.utils.gis_files.download()

nb_output_dir = pl.Path("./step3_mf6_api").resolve()
if not nb_output_dir.exists():
    nb_output_dir.mkdir()

pws_root = pws.constants.__pywatershed_root__
domain_dir = (pws_root.parent / "test_data/sagehen_5yr/").resolve()
mf6_domain_dir = (pws_root.parent / "test_data/sagehen_mf6").resolve()

# Get the MF6 dylibs from yesterdays nightly build

In [ ]:
nightly_build_dir = nb_output_dir / "mf6_nightly_build"
get_mf6_nightly_build(nightly_build_dir=nightly_build_dir, date="20240913")
if platform.system == "Windows":
    mf6_dll = nightly_build_dir / "bin/libmf6.dll"
else:
    mf6_dll = nightly_build_dir / "bin/libmf6.dylib"
assert mf6_dll.exists()

## Set up the run
### MF6 files staging
These need copied to the run directory

In [ ]:
files_dirs_to_cp = [
    "common",
    "sagehenmodel",
    "hru_weights.npz",
    "sagehen_postprocess_graphs.py",
    "prms_grid_v3-Copy1.nc",
]
rename = {
    "sagehenmodel": "run_dir",
    "sagehen_postprocess_graphs.py": "run_dir/sagehen_postprocess_graphs.py",
}
for name in files_dirs_to_cp:
    src = mf6_domain_dir / name
    if name in rename.keys():
        dst = nb_output_dir / rename[name]
    else:
        dst = nb_output_dir / name
    if not dst.exists():
        if src.is_dir():
            shutil.copytree(src, dst)
        else:
            shutil.copy(src, dst)

### pywatershed files
The pyawatershed files dont need copied, they are just used in memory. (THough they could be written out if you like).

In [ ]:
control_file = domain_dir / "sagehen_no_gw_cascades.control"
control = pws.Control.load_prms(control_file)

parameter_file = domain_dir / control.options["parameter_file"]
params = pws.parameters.PrmsParameters.load(parameter_file)
# TODO: THIS should be quiet with verbosity=0
params = pws.utils.preprocess_cascades.preprocess_cascade_params(
    control, params, verbosity=0
)

### Plot HRUs and the MF6 grid?

# Spatial mappings

In [ ]:
weights = np.load(nb_output_dir / "hru_weights.npz")

_HRU to UZF weights_

In [ ]:
print(weights["uzfw"].shape)
uzfw = weights["uzfw"]
nuzf_infilt = uzfw.shape[0]

_HRU to SFR weights_

In [ ]:
print(weights["sfrw"].shape)
sfrw = weights["sfrw"]

This indicates that here are 128 HRUs, 3386 gridcells, and 201 stream reaches. 

Define a function to map HRU values to MODFLOW 6 values, which is just a dot product.

In [ ]:
def hru2mf6(weights, values):
    return weights.dot(values)

#### Unit conversion factors

In [ ]:
m2ft = 3.28081
in2m = 1.0 / (12.0 * m2ft)
acre2m2 = 43560.0 / (m2ft * m2ft)

In [ ]:
hru_area_m2 = params.parameters["hru_area"] * acre2m2

### Run one-way coupled pywatershed and MODFLOW 6 models

#### Initialize pywatershed components

We establish and load the requisite inputs for pywawtershed. This parameter dictionary loaded from a pickle file below was adapted from a PRMS6 parameter file.

In [ ]:
run_dir = nb_output_dir / "run_dir"
assert run_dir.exists()
os.chdir(run_dir)
print("changing to run directory:\n", os.getcwd())

mf6_output_dir = run_dir / "output"

# This step is *critical* for MF6 initialization
if not mf6_output_dir.exists():
    mf6_output_dir.mkdir()

In [ ]:
control.options["input_dir"] = domain_dir
control.options["netcdf_output_dir"] = nb_output_dir / "pws_model_output"
control.options["calc_method"] = "numba"
control.options["budget_type"] = None
# TODO: THIS NEXT LINE SHOULD NOT NEED TO HAPPEN
control.options["netcdf_output_var_names"] = control.options[
    "netcdf_output_var_names"
].tolist()
control.options["netcdf_output_var_names"].remove("hru_hortn_cascflow")
control.options["netcdf_output_var_names"].append("hru_horton_cascflow")

#### Create arrays to save hydrology results
While pywatershed has NetCDF output available, the existing plots are setup to work with numpy arrays collected during the run and saved at the end. We follow this path for ease of reproducing the plots. This how custom output can be quite easily achieved with pywatershed.

In [ ]:
ntimes = int(control.n_times)
print("Number of days to simulate {}".format(ntimes))

prms_vars = [
    "ppt_out",
    "actet_out",
    "potet_out",
    "soilinfil_out",
    "runoff_out",
    "interflow_out",
]
prms_var_dict = {}
prms_var_dict["time_out"] = np.empty(ntimes, dtype="datetime64[s]")
for vv in prms_vars:
    prms_var_dict[vv] = np.zeros((ntimes, hru_area_m2.shape[0]), dtype=np.float64)

In [ ]:
prms_processes = [
    pws.PRMSSolarGeometry,
    pws.PRMSAtmosphere,
    pws.PRMSCanopy,
    pws.PRMSSnow,
    pws.PRMSRunoffCascadesNoDprst,
    pws.PRMSSoilzoneCascadesNoDprst,
]

prms_model = pws.Model(
    prms_processes,
    control=control,
    parameters=params,
)

#### Initialize MODFLOW 6

The dylib/DLL Initialization requires all inputs AND the output directory to exist.

In [ ]:
mf6_config_file = "mfsim.nam"
mf6 = ModflowApi(mf6_dll, working_directory=os.getcwd())
mf6.initialize(str(mf6_config_file))

Get information about the modflow model start and end times

In [ ]:
current_time = mf6.get_current_time()
end_time = mf6.get_end_time()
print(f"MF current_time: {current_time}, prms control.start_time: {control.start_time}")
print(f"MF end_time: {end_time}, prms control.n_times: {control.n_times}")

#### Get pointers to MODFLOW 6 variables

As shown in figure 1, pywatershed is sending the following fluxes to MF6

* SINF is surface infiltration to MF6's UZF package, this is mapped from groundwater recharge in pywatershed
* PET is unsatisfied potential evapotransipiration in MF6's UZF, this is also tracked through soilzone in pytwatershed
* RUNOFF to MF6's SFR comes from combined surface runoff and interflow from pywatershed.

To set these values on MF6, we need the to get the pointers into MF6 using its BMI interface.

In [ ]:
sim_name = "sagehenmodel"
mf6_var_model_dict = {"SINF": "UZF-1", "PET": "UZF-1", "RUNOFF": "SFR-1"}
mf6_vars = {}
for vv, mm in mf6_var_model_dict.items():
    mf6_vars[vv] = mf6.get_value_ptr(mf6.get_var_address(vv, sim_name.upper(), mm))

for vv, dd in mf6_vars.items():
    print(f"shape of {vv}: {dd.shape}")

The UZF model has 2 vertical layers, so the number of points in UZF is twice what was shown in the spatial weights mappings above.

#### Run the models

Now we run the model. We use pywatershed to control the simulation.

In [ ]:
n_time_steps = control.n_times  # redundant above?
for istep in range(n_time_steps):
    prms_model.advance()

    if control.current_dowy == 0:
        if istep > 0:
            print("\n")
        print(f"Water year: {control.current_year + 1}")

    msg = f"Day of water year: {str(control.current_dowy + 1).zfill(3)}"
    print(msg, end="\r")

    # run pywatershed
    prms_model.calculate()

    # calculate variables for output and coupling
    hru_ppt = prms_model.processes["PRMSAtmosphere"].hru_ppt.current

    potet = prms_model.processes["PRMSSoilzoneCascadesNoDprst"].potet
    actet = prms_model.processes["PRMSSoilzoneCascadesNoDprst"].hru_actet
    unused_pet = potet - actet

    soil_infil = (
        prms_model.processes["PRMSSoilzoneCascadesNoDprst"].ssres_in
        + prms_model.processes["PRMSSoilzoneCascadesNoDprst"].pref_flow_infil
    )
    recharge = (
        prms_model.processes["PRMSSoilzoneCascadesNoDprst"].ssr_to_gw
        + prms_model.processes["PRMSSoilzoneCascadesNoDprst"].soil_to_gw
    )

    sroff = prms_model.processes["PRMSRunoffCascadesNoDprst"].sroff
    interflow = prms_model.processes["PRMSSoilzoneCascadesNoDprst"].ssres_flow
    prms_ro = (sroff + interflow) * in2m * hru_area_m2

    # save PRMS results (converted to m3/d)
    prms_var_dict["time_out"][istep] = control.current_time
    prms_var_dict["ppt_out"][istep, :] = hru_ppt * in2m * hru_area_m2
    prms_var_dict["potet_out"][istep, :] = potet * in2m * hru_area_m2
    prms_var_dict["actet_out"][istep, :] = actet * in2m * hru_area_m2
    prms_var_dict["soilinfil_out"][istep, :] = soil_infil * in2m * hru_area_m2
    prms_var_dict["runoff_out"][istep, :] = sroff * in2m * hru_area_m2
    prms_var_dict["interflow_out"][istep, :] = interflow * in2m * hru_area_m2

    # Set MF6 pointers
    mf6_vars["RUNOFF"][:] = hru2mf6(sfrw, prms_ro)  # sroff + ssres_flow
    mf6_vars["SINF"][:nuzf_infilt] = (
        hru2mf6(uzfw, recharge) * in2m
    )  # ssr_to_gw + soil_to_gw
    mf6_vars["PET"][:nuzf_infilt] = hru2mf6(uzfw, unused_pet) * in2m  # potet - actet

    # run MODFLOW 6
    mf6.update()

try:
    mf6.finalize()
    prms_model.finalize()
    success = True
except:
    raise RuntimeError

# switch this to netcdf
fpth = "output/pywatershed_output.npz"
np.savez_compressed(
    fpth,
    time=prms_var_dict["time_out"],
    ppt=prms_var_dict["ppt_out"],
    potet=prms_var_dict["potet_out"],
    actet=prms_var_dict["actet_out"],
    infil=prms_var_dict["soilinfil_out"],
    runoff=prms_var_dict["runoff_out"],
    interflow=prms_var_dict["interflow_out"],
)

#### Finalize models
Clean up. 

#### Save hydrology output
Save the variables to file that were collected into memory from pywatershed.

### Plot

We spare the user the details of the plotting. 

In [ ]:
fig_out_dir = nb_output_dir / "figures"
fig_out_dir.mkdir(exist_ok=True)

run_dir = nb_output_dir / "run_dir"
os.chdir(run_dir)  # make sure

import sagehen_postprocess_graphs as graphs

In [ ]:
graphs.streamflow_fig()

In [ ]:
graphs.et_recharge_ppt_fig()

In [ ]:
graphs.gwf_uzf_storage_changes_fig()

In [ ]:
graphs.cumulative_streamflow_fig()

In [ ]:
graphs.composite_fig()

## Conclusions

This one-way coupling of pywatershed and MODFLOW 6 demonstrates how software interoperatbility standards can bridge silos with out creating new "code silos". 

This demonstration also highlights how modular models provide clear process conceptualizations and easy access to variables required for cross-discipline couplings. 

In the near future, pywatershed will expand to support the reproducing GSFLOW by implementing hydrologic "cascading flows" and implementing a 2-way couling with MF6. 


## References
* Hutton, E. W., Piper, M. D., & Tucker, G. E. (2020). The Basic Model Interface 2.0: A standard interface for coupling numerical models in the geosciences. Journal of Open Source Software, 5(51), 2317.
* Hughes, J. D., Russcher, M. J., Langevin, C. D., Morway, E. D., & McDonald, R. R. (2022). The MODFLOW Application Programming Interface for simulation control and software interoperability. Environmental Modelling & Software, 148, 105257.
* Langevin, C. D., Hughes, J. D., Banta, E. R., Niswonger, R. G., Panday, S., & Provost, A. M. (2017). Documentation for the MODFLOW 6 groundwater flow model (No. 6-A55). US Geological Survey.
* Markstrom, S. L., Niswonger, R. G., Regan, R. S., Prudic, D. E., & Barlow, P. M. (2008). GSFLOW-Coupled Ground-water and Surface-water FLOW model based on the integration of the Precipitation-Runoff Modeling System (PRMS) and the Modular Ground-Water Flow Model (MODFLOW-2005). US Geological Survey techniques and methods, 6, 240.
* Regan, R. S., Markstrom, S. L., Hay, L. E., Viger, R. J., Norton, P. A., Driscoll, J. M., & LaFontaine, J. H. (2018). Description of the national hydrologic model for use with the precipitation-runoff modeling system (prms) (No. 6-B9). US Geological Survey.
* Regan, R.S., Markstrom, S.L., LaFontaine, J.H., 2022, PRMS version 5.2.1: Precipitation-Runoff Modeling System (PRMS): U.S. Geological Survey Software Release, 02/10/2022.